In [8]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

# New Section

In [9]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [11]:
import json
f1 = open('./Datasets/typeA_B_train_dataset.json')
f2 = open('./Datasets/typeA_B_test_dataset.json')

TRAIN_DATA = json.load(f1)
TEST_DATA = json.load(f2)


In [12]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start,end,label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./Datasets/typeA_B_train_dataset.spacy")

100%|██████████| 92/92 [00:00<00:00, 6543.38it/s]


In [13]:
for text, annot in tqdm(TEST_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start,end,label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./Datasets/typeA_B_test_dataset.spacy")

100%|██████████| 15/15 [00:00<00:00, 6672.45it/s]

Skipping entity
Skipping entity


In [14]:
!python -m spacy train ./config.cfg --output ./model --paths.train ./Datasets/typeA_B_train_dataset.spacy --paths.dev ./Datasets/typeA_B_test_dataset.spacy

ℹ Saving to output directory: model
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     69.17   19.39   14.80   28.14    0.19
 36     200          1.86    888.39   99.66   99.33  100.00    1.00
 80     400          0.00      0.00   99.66   99.33  100.00    1.00
131     600          0.00      0.00   99.66   99.33  100.00    1.00
198     800          0.00      0.00   99.66   99.33  100.00    1.00
273    1000          0.00      0.00   99.66   99.33  100.00    1.00
^C


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [17]:
nlp_ner = spacy.load("./model/model-best")

In [ ]:
!ls /content/gdrive/MyDrive/Saved\ Models/typeA_model/

In [ ]:
nlp_ner.to_disk("/content/gdrive/MyDrive/Saved\ Models/typeA_model/")

OSError: [E050] Can't find model '/Users/om/Projects/Chrome Extensions/EchoGrid/model/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# Concatation trial

cmd = "Insert " + str(cells[0]) + " "+ str(values[0])

print(cmd)

In [ ]:
# code to create the final instruction for the Google Sheet API


for t in type:
  print(t)
  if t=="TYPE A":
    # the sentence is of INSERT

    cmd = "Insert " + str(cells[0]) + " "+ str(values[0])
    print(cmd)
  elif t=="TYPE B":
    # the sentence is of type delete
    cmd = "Delete " + str(cells[0])
    print(cmd)
